<a href="https://colab.research.google.com/github/CT2020Hypernym/hyponym_and_hyperonym/blob/Maria_spacy/spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy==2.2.3
!pip install pymorphy2

     |████████████████████████████████| 10.4MB 2.7MB/s 
     |████████████████████████████████| 122kB 40.4MB/s 
     |████████████████████████████████| 2.2MB 25.1MB/s 
     |████████████████████████████████| 3.7MB 47.6MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


     |████████████████████████████████| 51kB 1.4MB/s 
     |████████████████████████████████| 7.1MB 5.4MB/s 


In [0]:
!git clone https://github.com/buriy/spacy-ru.git

Cloning into 'spacy-ru'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 313 (delta 4), reused 7 (delta 2), pack-reused 301
Receiving objects: 100% (313/313), 117.34 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [0]:
import spacy
import random
import pandas as pd
from spacy.util import minibatch, compounding

from spacy.lang.ru import Russian

n_texts = 65030
n_iter = 20

def dataParser():
  #maybe create new file and work with it
  data = pd.read_csv('./pos_hypernymy.csv')
  data['coef'] = '1'
  data['concat'] = data['child_id'] + ' ' + data['parent_id']
  train_pos_df = data[['concat', 'coef']].dropna()
  data = pd.read_csv('./neg_hypernymy.csv')
  data['coef'] = '1'
  data['concat'] = data['child_id'] + ' ' + data['parent_id']
  train_neg_df = data[['concat', 'coef']].dropna()
  train_df=train_pos_df.append(train_neg_df)

  train_df['tuples'] = train_df.apply(
      lambda row: (row['concat'],row['coef']), axis=1)
  train = train_df['tuples'].tolist()
  return train

def load_data(limit=0, split=0.8):
    train_data = dataParser()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{1: bool(y), 0: not bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

if __name__ == '__main__':
    #load model (ru)
    #nlp = spacy.load('spacy-ru/ru2')
    nlp = Russian()
    nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

    if 'textcat' not in nlp.pipe_names:
      textcat = nlp.create_pipe('textcat')
      nlp.add_pipe(textcat, last=True)
    else:
      textcat = nlp.get_pipe('textcat')

    #categories
    textcat.add_label("1") #with hypernymy
    textcat.add_label("0") #without

    #load dataset

    (train_texts, train_cats), (dev_texts, dev_cats) = load_data()
    train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
    print("Using {} examples ({} training, {} evaluation)"
      .format(n_texts, len(train_texts), len(dev_texts)))

    #train model
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                          losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print(
                "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                )
            )
    print("Training the model ended.")

    #save the model
    output_dir = %pwd
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


Using 65030 examples (52024 training, 13006 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000
0.000	0.000	0.000	0.000


In [0]:
!pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git

  Cloning https://github.com/aatimofeev/spacy_russian_tokenizer.git to /tmp/pip-req-build-h1phe9nd
  Running command git clone -q https://github.com/aatimofeev/spacy_russian_tokenizer.git /tmp/pip-req-build-h1phe9nd
  Created wheel for spacy-russian-tokenizer: filename=spacy_russian_tokenizer-0.1.1-cp36-none-any.whl size=12674 sha256=738531e500c78af1c42974cf516eddfa40076d469e0a23b54d17cf3e8c7e7a02
  Stored in directory: /tmp/pip-ephem-wheel-cache-yeur6279/wheels/37/3b/bb/cfe712f7c0b78cd08f4a2ef122d17748baf9d4bebecf2e5a54
Successfully built spacy-russian-tokenizer


In [0]:
!git clone https://github.com/dialogue-evaluation/taxonomy-enrichment.git

Cloning into 'taxonomy-enrichment'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 114 (delta 16), reused 33 (delta 6), pack-reused 63
Receiving objects: 100% (114/114), 17.72 MiB | 8.04 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [18]:
import spacy
import pandas as pd

nlp = spacy.load("/content")
df = pd.read_csv("./taxonomy-enrichment/data/public_test/nouns_public.tsv", sep="\t")
#print(df.head())

doc = nlp('анкетирование сбор')
print(doc.cats)
doc = nlp('индиго пигмент')
print(doc.cats)
doc = nlp('анкетирование медведь')
print(doc.cats)

#doc.cats.items()

#print(df['АБДОМИНОПЛАСТИКА'][0]) #на самом деле подаем слово и его кандидаты, потом их сортируем и берем десять первых


{'1': 0.7040451765060425, '0': 0.8136601448059082}
{'1': 0.8838895559310913, '0': 0.8772980570793152}
{'1': 0.7902513146400452, '0': 0.5155986547470093}
